# Neurog Internship: (Task 1 - Fetching, Cleaning, Manipulating & Saving Data) 

## All Libraries

In [252]:
# Binance Package
from binance.exceptions import BinanceAPIException
from binance.futures import Futures 

# For Manipulating Date
import datetime
import time

# Other Necessary Libraries
import pandas as pd
import random
import copy

# Libraries to Make Code Cleaner
from typing import Union
from typing import List

## All Declarations

In [257]:
# Defining my API key & my secret key as well
API_Secret = "Your_API_Secret"
API_Key    = "Your_API_Key"

# Defining the start time from where the data needs to be collected
start_str = '2020-01-01 00:00:00'

# Define the interval
interval = Client.KLINE_INTERVAL_1MINUTE  # 1 minute interval

# The coin that you need the data for
symbol = "BTCUSDT"

## Utility Functions

In [262]:
# Function to change the fetched data into a dataframe
def ohlc_to_dataframe(data: list) -> pd.DataFrame:
    """
    Convert OHLC (Open, High, Low, Close) data to a pandas DataFrame with correct column names and index.

    Args:
    - data (list of lists): List containing OHLC data where each element is a list representing a row of OHLC data.

    Returns:
    - df (pd.DataFrame): DataFrame with OHLC data, indexed by 'Open time (1M)' & columns for 'Open', 'High', 'Low',
      'Close', 'Volume'.

    """
    # Define column names based on Binance API response
    columns = [
        'Open time (1M)', 'Open', 'High', 'Low', 'Close', 'Volume', 
        'Close time', 'Quote asset volume', 'Number of trades', 
        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
    ]
    
    # Create DataFrame from data with specified column names
    df = pd.DataFrame(data, columns = columns)
    
    # Convert 'Open time (1M)' to datetime & set as index
    df['Open time (1M)'] = pd.to_datetime(df['Open time (1M)'], unit = 'ms')
    df.set_index('Open time (1M)', inplace = True)
    
    # Drop unnecessary columns
    df.drop(columns = ['Close time', 'Ignore'], inplace = True)
    
    # Convert all columns to numeric, coercing errors to NaN
    df = df.apply(pd.to_numeric, errors = 'coerce')
    
    return df


# Functions converts the dataframe into any given time frame.
def convert_1m_to_any_timeframe(df: pd.DataFrame, timeframe: str) -> pd.DataFrame:
    """
    Convert a DataFrame of 1-minute OHLC data to any given timeframe.

    Args:
    - df (pd.DataFrame): DataFrame containing 1-minute OHLC data. 
      The DataFrame should have a datetime index & columns ['Open', 'High', 'Low', 'Close', 'Volume'].
    - timeframe (str): The desired timeframe to resample the data to (e.g., '1H' for 1 hour, '1D' for 1 day).

    Returns:
    - pd.DataFrame: Resampled DataFrame with OHLC data in the specified timeframe. The index will be renamed to
      reflect the new timeframe.

    Example:
    ```
    resampled_df = convert_1m_to_any_timeframe(ohlc_df, '1H')
    ```
    """
    # Ensure the DataFrame index is of datetime type
    df.index = pd.to_datetime(df.index)
    
    # Try resampling the data to the desired timeframe
    try:
        df_resampled = df.resample(timeframe).agg({
            'Open': 'first',  # Take the first 'Open' value in the timeframe
            'High': 'max',    # Take the maximum 'High' value in the timeframe
            'Low': 'min',     # Take the minimum 'Low' value in the timeframe
            'Close': 'last',  # Take the last 'Close' value in the timeframe
            'Volume': 'sum'  # Take the mean 'Volume' value in the timeframe
        })
    except Exception as e:
        print(f"An error occurred while resampling! Error message: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

    # Rename the index to reflect the new timeframe
    df_resampled.index.rename(f'Open time ({timeframe})', inplace = True)
    
    return df_resampled


# Function that saves the dataframe into a csv given the file name
def save_to_csv(df: pd.DataFrame, filename: str):
    """
    Save DataFrame to a CSV file in the 'ohlc' folder within the 'data' directory.
    Creates 'data' and 'ohlc' directories if they do not exist.

    Args:
    - df (pd.DataFrame): DataFrame containing the data to be saved.
    - filename (str): Name of the CSV file to save.

    Returns:
    - None

    Raises:
    - ValueError: If the DataFrame (`df`) is empty or `filename` is empty.

    """
    if df.empty:
        raise ValueError("DataFrame is empty. Nothing to save.")
    
    if not filename:
        raise ValueError("Invalid filename provided.")

    # Construct the full path
    data_dir = os.path.join("..", "data")
    ohlc_dir = os.path.join(data_dir, "ohlc")
    full_path = os.path.join(ohlc_dir, filename)
    
    # Create directories if they do not exist
    os.makedirs(ohlc_dir, exist_ok=True)
    
    try:
        df.to_csv(full_path, index=True)
        print(f"Data saved to {full_path}")
    except Exception as e:
        print(f"Failed to save data to {full_path}. Error: {e}")

## Function to Fetch Data From Binance API

In [264]:
# Function to get & return OHLC data for a given COIN (symbo)
def fetch_ohlc_data(symbol: str, interval: str, start_str: str, end_str: Union[str, None] = None) -> List[List[float]]:
    """
    Fetch historical klines (candlestick) data from Binance Futures API.

    Args:
    - symbol (str): The trading pair symbol (e.g., 'BTCUSDT').
    - interval (str): The interval of the kline (e.g., '1m' for 1 minute, '1h' for 1 hour).
    - start_str (str): The start date & time in the format "%Y-%m-%d %H:%M:%S".
    - end_str (str or None, optional): The end date & time in the format "%Y-%m-%d %H:%M:%S". Defaults to None.

    Returns:
    - klines (List[List[float]]): A list of kline data where each element is a list representing a kline.

    """
    klines = []
    limit  = 1000  # Maximum number of records per request

    # Convert start_str & end_str to datetime objects
    start_dt = datetime.datetime.strptime(start_str, "%Y-%m-%d %H:%M:%S")
    end_dt   = datetime.datetime.now() if end_str is None else datetime.datetime.strptime(end_str, "%Y-%m-%d %H:%M:%S")

    # Loop until we fetch all required data from start_dt to end_dt
    while start_dt < end_dt:
        start_ts = int(start_dt.timestamp() * 1000)

        # Fetch klines data from Binance API
        temp_klines = client.klines(
            symbol    = symbol,
            interval  = interval,
            startTime = start_ts,
            limit     = limit
        )

        # Check if no more data is available
        if not temp_klines:
            break
        
        # Append fetched klines to the result list
        klines = klines + temp_klines
        
        # Update start_dt to the next timestamp after the last fetched kline
        start_dt = datetime.datetime.fromtimestamp(temp_klines[-1][0] / 1000.0) + datetime.timedelta(minutes = 1)
        
        # Break loop if fetched data is less than the limit
        if len(temp_klines) < limit:
            break
        
        # Sleep to avoid hitting rate limits
        time.sleep(1)

    return klines

## Main

### Setting Up The Client Using Our Keys

In [45]:
# Call the constructor of futures
client = Futures()
client = Futures(key = API_Key, secret = API_Secret, base_url = "https://testnet.binancefuture.com")

{'serverTime': 1719293160136}


### Getting The Account Info & Making Sure Everything Is Working Just Fine!

In [46]:
# Get account information
try:
    account_info = client.account()
    print("Account Information:")
    print(account_info)
except BinanceAPIException as e:
    print(f"Binance API Exception: {e}")
except Exception as e:
    print(f"An unexpected error occurred while fetching account information: {e}")

Account Information:
{'feeTier': 0, 'canTrade': True, 'canDeposit': True, 'canWithdraw': True, 'feeBurn': True, 'tradeGroupId': -1, 'updateTime': 0, 'multiAssetsMargin': False, 'totalInitialMargin': '0.00000000', 'totalMaintMargin': '0.00000000', 'totalWalletBalance': '15000.00000000', 'totalUnrealizedProfit': '0.00000000', 'totalMarginBalance': '15000.00000000', 'totalPositionInitialMargin': '0.00000000', 'totalOpenOrderInitialMargin': '0.00000000', 'totalCrossWalletBalance': '15000.00000000', 'totalCrossUnPnl': '0.00000000', 'availableBalance': '15000.00000000', 'maxWithdrawAmount': '15000.00000000', 'assets': [{'asset': 'FDUSD', 'walletBalance': '0.00000000', 'unrealizedProfit': '0.00000000', 'marginBalance': '0.00000000', 'maintMargin': '0.00000000', 'initialMargin': '0.00000000', 'positionInitialMargin': '0.00000000', 'openOrderInitialMargin': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.000

### Calling The Function To Get The Data From The API!

In [96]:
## Uncomment this cell to fetch the data again!
## Fetch and print the OHLC data
# all_data = fetch_ohlc_data(symbol, interval, start_str)

In [97]:
print(len(all_data))

2357892


### Converting The Fetched Data To Pandas DataFrame

In [233]:
# Convert to DataFrame
df_ohlc = ohlc_to_dataframe(all_data)
df_ohlc

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Open time (1M),,,,,,,,,
2019-12-31 19:00:00,7169.71,7169.71,7165.44,7167.83,3509.860,2.515594e+07,139,1025.400,7.348382e+06
2019-12-31 19:01:00,7167.83,7168.28,7158.66,7159.95,3821.170,2.737640e+07,155,1571.990,1.126101e+07
2019-12-31 19:02:00,7161.03,7165.40,7161.03,7162.46,3041.710,2.178758e+07,122,1439.790,1.031332e+07
2019-12-31 19:03:00,7161.74,7164.27,7160.30,7161.03,3682.650,2.637680e+07,147,1557.630,1.115700e+07
2019-12-31 19:04:00,7161.03,7164.25,7160.15,7160.15,2936.690,2.103310e+07,113,1407.430,1.007968e+07
...,...,...,...,...,...,...,...,...,...
2024-06-25 07:07:00,63742.80,63742.80,62743.60,62743.60,1.216,7.736040e+04,12,1.042,6.642000e+04
2024-06-25 07:08:00,62743.60,63742.80,62652.60,62655.00,0.726,4.555072e+04,12,0.056,3.568624e+03
2024-06-25 07:09:00,62655.00,63742.80,62652.60,62652.60,2.820,1.779314e+05,26,2.690,1.697798e+05


### The Open Time Is Somehow 5 Hours Shifted Backwards. Correcting The Time Shift!

In [234]:
# I dont know why, but needed to shift time by 5 hours..
shift_amount = pd.Timedelta('5H')

# Shift the index
df_ohlc.index = df_ohlc.index.map(lambda x: x + shift_amount)

df_ohlc

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Open time (1M),,,,,,,,,
2020-01-01 00:00:00,7169.71,7169.71,7165.44,7167.83,3509.860,2.515594e+07,139,1025.400,7.348382e+06
2020-01-01 00:01:00,7167.83,7168.28,7158.66,7159.95,3821.170,2.737640e+07,155,1571.990,1.126101e+07
2020-01-01 00:02:00,7161.03,7165.40,7161.03,7162.46,3041.710,2.178758e+07,122,1439.790,1.031332e+07
2020-01-01 00:03:00,7161.74,7164.27,7160.30,7161.03,3682.650,2.637680e+07,147,1557.630,1.115700e+07
2020-01-01 00:04:00,7161.03,7164.25,7160.15,7160.15,2936.690,2.103310e+07,113,1407.430,1.007968e+07
...,...,...,...,...,...,...,...,...,...
2024-06-25 12:07:00,63742.80,63742.80,62743.60,62743.60,1.216,7.736040e+04,12,1.042,6.642000e+04
2024-06-25 12:08:00,62743.60,63742.80,62652.60,62655.00,0.726,4.555072e+04,12,0.056,3.568624e+03
2024-06-25 12:09:00,62655.00,63742.80,62652.60,62652.60,2.820,1.779314e+05,26,2.690,1.697798e+05


### Sampling Random 5 Rows & Manually Checking If Data Is Correct Or Not (It's Correct!)

In [235]:
# To manually check if values are correct or not
df_ohlc.sample(n = 5)

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Open time (1M),,,,,,,,,
2020-12-07 00:22:00,19128.93,19129.31,19127.74,19128.93,30.726,5.877537e+05,16,16.595,3.174466e+05
2022-02-09 22:41:00,44176.30,44190.50,44157.70,44161.90,101.570,4.486205e+06,19,31.654,1.398032e+06
2024-03-30 03:35:00,69667.00,70000.00,69667.00,69667.00,1.556,1.084398e+05,4,0.114,7.980000e+03
2020-03-14 19:48:00,5345.44,5349.34,5343.63,5344.26,3743.168,2.001001e+07,167,1268.058,6.778809e+06
2023-01-19 21:04:00,20865.90,20865.90,20840.30,20846.40,116.560,2.430262e+06,100,54.520,1.136822e+06


### Saving The Data To CSV

In [239]:
save_to_csv(df_ohlc, "BTCUSDT_1_Minute_From_2020_Till_Now.csv")

Data saved to BTCUSDT_1_Minute_From_2020_to_6_25_2024


### Filtering Out The Necessary Columns

In [240]:
necessary_cols_df = df_ohlc[["Open", "High", "Low", "Close", "Volume"]]
necessary_cols_df

,Open,High,Low,Close,Volume
Open time (1M),,,,,
2020-01-01 00:00:00,7169.71,7169.71,7165.44,7167.83,3509.860
2020-01-01 00:01:00,7167.83,7168.28,7158.66,7159.95,3821.170
2020-01-01 00:02:00,7161.03,7165.40,7161.03,7162.46,3041.710
2020-01-01 00:03:00,7161.74,7164.27,7160.30,7161.03,3682.650
2020-01-01 00:04:00,7161.03,7164.25,7160.15,7160.15,2936.690
...,...,...,...,...,...
2024-06-25 12:07:00,63742.80,63742.80,62743.60,62743.60,1.216
2024-06-25 12:08:00,62743.60,63742.80,62652.60,62655.00,0.726
2024-06-25 12:09:00,62655.00,63742.80,62652.60,62652.60,2.820


### Saving The Filtered Column DataFrame To CSV As Well!

In [266]:
save_to_csv(necessary_cols_df, "BTCUSDT_1_Minute_From_2020_To_Latest_Filtered.csv")

Data saved to BTCUSDT_1_Minute_From_2020_to_6_25_2024_Filtered


### Converting The DataFrame From 1 Minute Timeframe To 1 Hour Timeframe

In [267]:
one_hour_ohlc_df = convert_1m_to_any_timeframe(copy.deepcopy(necessary_cols_df), "1H")
one_hour_ohlc_df

,Open,High,Low,Close,Volume
Open time (1H),,,,,
2020-01-01 00:00:00,7169.71,7183.07,7156.65,7174.14,3200.843000
2020-01-01 01:00:00,7173.42,7186.36,7164.51,7175.44,3262.390850
2020-01-01 02:00:00,7175.44,7188.52,7168.78,7187.44,3616.975000
2020-01-01 03:00:00,7187.20,7207.23,7181.09,7202.48,3632.039517
2020-01-01 04:00:00,7202.48,7209.48,7184.34,7195.54,3505.551667
...,...,...,...,...,...
2024-06-25 08:00:00,61965.80,63998.80,60531.00,61592.20,19.264567
2024-06-25 09:00:00,62999.70,63995.90,60608.00,61653.50,3.120217
2024-06-25 10:00:00,61302.80,63996.00,60802.00,63892.00,14.673217


### Converting THe DataFrame From 1 Minute Timeframe to 1 Day Timeframe

In [265]:
one_day_ohlc_df = convert_1m_to_any_timeframe(copy.deepcopy(necessary_cols_df), "1D")
one_day_ohlc_df

,Open,High,Low,Close,Volume
Open time (1D),,,,,
2020-01-01,7169.71,9592.00,6871.45,7242.21,50472.792781
2020-01-02,7242.21,7243.41,6934.53,6975.70,3348.620363
2020-01-03,6975.70,7600.00,6871.45,7342.00,3584.646061
2020-01-04,7342.00,7400.00,7260.00,7305.62,3105.244638
2020-01-05,7305.68,7492.05,7260.00,7438.62,2969.803410
...,...,...,...,...,...
2024-06-21,65788.00,71341.20,64100.00,66398.30,16.093789
2024-06-22,66494.90,67625.40,64983.20,65865.00,5.460945
2024-06-23,65806.10,65979.90,64999.50,65400.80,3.795993


# ***More Later!***